In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
from tabulate import tabulate
from scipy.stats import f

class GaugeRRApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Gauge R&R Analysis")
        
        self.create_widgets()
    
    def create_widgets(self):
        # Create GUI widgets
        self.file_label = ttk.Label(self.master, text="Select Excel File:")
        self.file_label.grid(row=0, column=0, padx=10, pady=5, sticky="w")
        
        self.file_entry = ttk.Entry(self.master, width=40)
        self.file_entry.grid(row=0, column=1, padx=10, pady=5, sticky="w")
        
        self.browse_button = ttk.Button(self.master, text="Browse", command=self.browse_file)
        self.browse_button.grid(row=0, column=2, padx=10, pady=5, sticky="w")
        
        self.operator_label = ttk.Label(self.master, text="Operator Column:")
        self.operator_label.grid(row=1, column=0, padx=10, pady=5, sticky="w")
        
        self.operator_var = tk.StringVar(self.master)
        self.operator_dropdown = ttk.Combobox(self.master, textvariable=self.operator_var, width=20)
        self.operator_dropdown.grid(row=1, column=1, padx=10, pady=5, sticky="w")
        
        self.part_label = ttk.Label(self.master, text="Part Column:")
        self.part_label.grid(row=2, column=0, padx=10, pady=5, sticky="w")
        
        self.part_var = tk.StringVar(self.master)
        self.part_dropdown = ttk.Combobox(self.master, textvariable=self.part_var, width=20)
        self.part_dropdown.grid(row=2, column=1, padx=10, pady=5, sticky="w")
        
        self.trials_label = ttk.Label(self.master, text="Trials Column:")
        self.trials_label.grid(row=3, column=0, padx=10, pady=5, sticky="w")
        
        self.trials_var = tk.StringVar(self.master)
        self.trials_dropdown = ttk.Combobox(self.master, textvariable=self.trials_var, width=20)
        self.trials_dropdown.grid(row=3, column=1, padx=10, pady=5, sticky="w")
        
        self.measurement_label = ttk.Label(self.master, text="Measurement Column:")
        self.measurement_label.grid(row=4, column=0, padx=10, pady=5, sticky="w")
        
        self.measurement_var = tk.StringVar(self.master)
        self.measurement_dropdown = ttk.Combobox(self.master, textvariable=self.measurement_var, width=20)
        self.measurement_dropdown.grid(row=4, column=1, padx=10, pady=5, sticky="w")
       
        self.run_button = ttk.Button(self.master, text="Run Analysis", command=self.run_analysis)
        self.run_button.grid(row=6, column=0, columnspan=3, padx=10, pady=10, sticky="we")
        
        
    def browse_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx;*.xls")])
        if file_path:
            self.file_entry.delete(0, tk.END)
            self.file_entry.insert(0, file_path)
            self.load_excel_columns(file_path)  # Call load_excel_columns function

    
    def load_excel_columns(self, file_path):
        df = pd.read_excel(file_path)
        columns = df.columns.tolist()
        self.operator_dropdown['values'] = columns
        self.part_dropdown['values'] = columns
        self.trials_dropdown['values'] = columns
        self.measurement_dropdown['values'] = columns

    
    def run_analysis(self):
        file_path = self.file_entry.get()
        
        if not file_path:
            print("Error", "Please select an Excel file.")
            return
        
        try:
            df = pd.read_excel(file_path)
            operator_column = self.operator_var.get()
            part_column = self.part_var.get()
            trials_column = self.trials_var.get()
            measurement_column = self.measurement_var.get()

            if not all([operator_column, part_column, trials_column, measurement_column]):
                print("Please select all columns.")
                return
            
            # Perform ANOVA
            results = self.perform_crossed_anova(df, operator_column, part_column, trials_column, measurement_column)
            
            if results:
                # Retrieve p-value for Operator x Part
                p_operatorxpart = results.get("p_operatorxpart")

                print("P-value: operator x part", p_operatorxpart)


                # Set threshold for deciding which tables to print
                threshold = 0.05

                if p_operatorxpart is not None and p_operatorxpart > threshold:
                    print("P-value for operator x part is ",p_operatorxpart," which is less than threshold ", threshold,".\n"
                          "Operator x Part interaction is not statistically significant")
                    # Print only tables for "wo_int"
                    self.print_wo_int(results)
                else:
                    # Print tables for both "with Interaction"
                    print("P-value for operator x part is ",p_operatorxpart," which is greater than threshold ", threshold,".\n"
                          "Operator x Part interaction is statistically significant")
                    self.print_w_int(results)


        except Exception as e:
            print(f"An error occurred: {str(e)}")


    
    def perform_crossed_anova(self, df, operator_column, part_column, trials_column, measurement_column):
        try:
            # Retrieve selected column names from dropdown menus
            operator_column = self.operator_var.get()
            part_column = self.part_var.get()
            trials_column = self.trials_var.get()
            measurement_column = self.measurement_var.get()

            # Check if all columns are selected
            if not operator_column or not part_column or not trials_column or not measurement_column:
                messagebox.showerror("Error", "Please select all columns.")
                return None

            # Calculate the operator sum of squares
            operator_avg = df.groupby(operator_column)[measurement_column].mean()
            overall_avg = df[measurement_column].mean()
            SS_operator = ((operator_avg - overall_avg) ** 2).sum() * len(df[part_column].unique()) * len(df[trials_column].unique())

            # Step 2: Compute the parts sum of squares
            part_avg = df.groupby(part_column)[measurement_column].mean()

            SS_part = ((part_avg - overall_avg) ** 2).sum() * len(df[operator_column].unique()) * len(df[trials_column].unique())


            # Calculate the total sum of squares
            SS_total = ((df[measurement_column] - overall_avg) ** 2).sum()


            # Compute Equipment within the sum of squares
            SS_equipment = 0
            for part in df[part_column].unique():
                for operator in df[operator_column].unique():
                    part_operator_avg = df[(df[part_column] == part) & (df[operator_column] == operator)][measurement_column].mean()
                    squared_deviation = ((df[(df[part_column] == part) & (df[operator_column] == operator)][measurement_column] - part_operator_avg) ** 2).sum()
                    SS_equipment += squared_deviation


            # Find the interaction sum of squares
            SS_operatorxpart = SS_total - (SS_operator + SS_part + SS_equipment)

            SS_repeatability_w_int = SS_equipment

            SS_repeatability_wo_int = SS_operatorxpart + SS_repeatability_w_int

            # Evaluate repeatability, operator, part, and interaction variance
            n_operators = len(df[operator_column].unique())
            n_parts = len(df[part_column].unique())
            n_trials = len(df[trials_column].unique())
            
            df_operator = n_operators - 1
            df_part = n_parts - 1
            df_operatorxpart = df_operator * df_part
            df_repeatability_w_int = n_operators * n_parts * (n_trials-1)
            df_repeatability_wo_int = df_repeatability_w_int + df_operatorxpart

            MS_repeatability_w_int = round(SS_repeatability_w_int/df_repeatability_w_int, 3)
            MS_repeatability_wo_int = round(SS_repeatability_wo_int/df_repeatability_wo_int, 3)
            MS_operatorxpart = round(SS_operatorxpart / df_operatorxpart, 3)
            MS_part = round(SS_part / df_part, 3)
            MS_equipment = round(SS_equipment/(n_operators*n_parts*(n_trials-1)), 3)
            MS_operator = round(SS_operator / df_operator, 3)

            F_operator = round(MS_operator/MS_operatorxpart, 3)
            F_part = round(MS_part/MS_operatorxpart, 3)
            F_operatorxpart = round(MS_operatorxpart/MS_equipment, 3)


            alpha = 0.05
            
            p_operator = round(1-f.cdf(abs(F_operator), df_operator, df_operatorxpart), 3)
            p_part = round(1-f.cdf(abs(F_part), df_part, df_operatorxpart), 3)
            p_operatorxpart = round(1-f.cdf(abs(F_operatorxpart), df_operatorxpart, df_repeatability_w_int), 3)
            
            table_data = [
                ["Operator", df_operator, SS_operator, MS_operator, F_operator, p_operator],
                ["Part", df_part, SS_part, MS_part, F_part, p_part],
                ["Interaction (operator x part)", df_operatorxpart, SS_operatorxpart, MS_operatorxpart, F_operatorxpart, p_operatorxpart],
                ["Repeatability (with Interaction)", df_repeatability_w_int, SS_repeatability_w_int, MS_repeatability_w_int, ""],
                ["Repeatability (wo Interaction)", df_repeatability_wo_int, SS_repeatability_wo_int, MS_repeatability_wo_int, ""]
            ]

            headers = ["Source", "DF", "Sum of Square", "Mean Square", "F", "p-value"]
            print(tabulate(table_data, headers=headers, tablefmt="pretty"))

            sigma2_wo_int = round(MS_repeatability_wo_int, 3)
            sigma2_w_int = round(MS_repeatability_w_int, 3)
            sigma2_operatorxpart_wo_int = round((MS_operatorxpart - sigma2_wo_int**2)/n_trials, 3)
            sigma2_operatorxpart_w_int = round((MS_operatorxpart - sigma2_w_int**2)/n_trials, 3)
            sigma2_part = round((MS_part-MS_operatorxpart)/(n_trials*n_operators), 3)
            sigma2_operator = round((MS_operator - MS_operatorxpart)/(n_trials*n_parts), 3)

            SD_operator = round(sigma2_operator**.5, 3)
            SD_p2p = round(sigma2_part**.5, 3)
            
            # Variables related to w_int
            variance_totalGRR_w_int = round(sigma2_w_int + sigma2_operator, 3)
            variance_reproducibility_w_int = round(sigma2_operator + sigma2_operatorxpart_w_int, 3)
            var_tot_w_int = round(sigma2_w_int + sigma2_operator + sigma2_operatorxpart_w_int + sigma2_part, 3)
            pct_var_contb_totalGRR_w_int = round(variance_totalGRR_w_int / var_tot_w_int, 3)
            pct_var_contb_repeatability_w_int = round(sigma2_w_int / var_tot_w_int, 3)
            pct_var_contb_reproducibility_w_int = round(variance_reproducibility_w_int / var_tot_w_int, 3)
            pct_var_contb_operator_w_int = round(sigma2_operator / var_tot_w_int, 3)
            pct_var_contb_operatorxpart_w_int = round(sigma2_operatorxpart_w_int / var_tot_w_int, 3)
            pct_var_contb_p2p_w_int = round(sigma2_part / var_tot_w_int, 3)
            SD_totalGRR_w_int = round(variance_totalGRR_w_int ** 0.5, 3)
            SD_repeatability_w_int = round(sigma2_w_int ** 0.5, 3)
            SD_reproducibility_w_int = round(variance_reproducibility_w_int ** 0.5, 3)
            SD_operatorxpart_w_int = round(sigma2_operatorxpart_w_int ** 0.5, 3)
            SD_total_w_int = round(var_tot_w_int ** 0.5, 3)
            pct_SV_totalGRR_w_int = round(SD_totalGRR_w_int / SD_total_w_int, 3) * 100
            pct_SV_repeatability_w_int = round(SD_repeatability_w_int / SD_total_w_int, 3) * 100
            pct_SV_reproducibility_w_int = round(SD_reproducibility_w_int / SD_total_w_int, 3) * 100
            pct_SV_operator_w_int = round(SD_operator / SD_total_w_int, 3) * 100
            pct_SV_operatorxpart_w_int = round(SD_operatorxpart_w_int / SD_total_w_int, 3) * 100
            pct_SV_p2p_w_int = round(SD_p2p / SD_total_w_int, 3) * 100

            # Variables related to wo_int
            variance_totalGRR_wo_int = round(sigma2_wo_int + sigma2_operator, 3)
            variance_reproducibility_wo_int = round(sigma2_operator + sigma2_operatorxpart_wo_int, 3)
            var_tot_wo_int = round(sigma2_wo_int + sigma2_operator + sigma2_operatorxpart_wo_int + sigma2_part, 3)
            pct_var_contb_totalGRR_wo_int = round(variance_totalGRR_wo_int / var_tot_wo_int, 3)
            pct_var_contb_repeatability_wo_int = round(sigma2_wo_int / var_tot_wo_int, 3)
            pct_var_contb_reproducibility_wo_int = round(variance_reproducibility_wo_int / var_tot_wo_int, 3)
            pct_var_contb_operator_wo_int = round(sigma2_operator / var_tot_wo_int, 3)
            pct_var_contb_operatorxpart_wo_int = round(sigma2_operatorxpart_wo_int / var_tot_wo_int, 3)
            pct_var_contb_p2p_wo_int = round(sigma2_part / var_tot_wo_int, 3)
            SD_totalGRR_wo_int = round(variance_totalGRR_wo_int ** 0.5, 3)
            SD_repeatability_wo_int = round(sigma2_wo_int ** 0.5, 3)
            SD_reproducibility_wo_int = round(variance_reproducibility_wo_int ** 0.5, 3)
            SD_operatorxpart_wo_int = round(sigma2_operatorxpart_wo_int ** 0.5, 3)
            SD_total_wo_int = round(var_tot_wo_int ** 0.5, 3)
            pct_SV_totalGRR_wo_int = round(SD_totalGRR_wo_int / SD_total_wo_int, 3) * 100
            pct_SV_repeatability_wo_int = round(SD_repeatability_wo_int / SD_total_wo_int, 3) * 100
            pct_SV_reproducibility_wo_int = round(SD_reproducibility_wo_int / SD_total_wo_int, 3) * 100
            pct_SV_operator_wo_int = round(SD_operator / SD_total_wo_int, 3) * 100
            pct_SV_operatorxpart_wo_int = round(SD_operatorxpart_wo_int / SD_total_wo_int, 3) * 100
            pct_SV_p2p_wo_int = round(SD_p2p / SD_total_wo_int, 3) * 100


        

            return {
            
            # General variables
            "SD_operator": SD_operator,
            "SD_p2p": SD_p2p,
            "p_operatorxpart": p_operatorxpart,
            
            # Variables for without interaction
            "sigma2_wo_int": sigma2_wo_int,
            "sigma2_operatorxpart_wo_int": sigma2_operatorxpart_wo_int,
            "sigma2_part": sigma2_part,
            "sigma2_operator": sigma2_operator,
            "variance_totalGRR_wo_int": variance_totalGRR_wo_int,
            "variance_reproducibility_wo_int": variance_reproducibility_wo_int,
            "var_tot_wo_int": var_tot_wo_int,
            "pct_var_contb_totalGRR_wo_int": pct_var_contb_totalGRR_wo_int,
            "pct_var_contb_repeatability_wo_int": pct_var_contb_repeatability_wo_int,
            "pct_var_contb_reproducibility_wo_int": pct_var_contb_reproducibility_wo_int,
            "pct_var_contb_operator_wo_int": pct_var_contb_operator_wo_int,
            "pct_var_contb_operatorxpart_wo_int": pct_var_contb_operatorxpart_wo_int,
            "pct_var_contb_p2p_wo_int": pct_var_contb_p2p_wo_int,
            "SD_totalGRR_wo_int": SD_totalGRR_wo_int,
            "SD_repeatability_wo_int": SD_repeatability_wo_int,
            "SD_reproducibility_wo_int": SD_reproducibility_wo_int,
            "SD_operatorxpart_wo_int": SD_operatorxpart_wo_int,
            "SD_total_wo_int": SD_total_wo_int,
            "pct_SV_totalGRR_wo_int": pct_SV_totalGRR_wo_int,
            "pct_SV_repeatability_wo_int": pct_SV_repeatability_wo_int,
            "pct_SV_reproducibility_wo_int": pct_SV_reproducibility_wo_int,
            "pct_SV_operator_wo_int": pct_SV_operator_wo_int,
            "pct_SV_operatorxpart_wo_int": pct_SV_operatorxpart_wo_int,
            "pct_SV_p2p_wo_int": pct_SV_p2p_wo_int,
            
            #variables for with interaction
            "sigma2_w_int": sigma2_w_int,
            "sigma2_operatorxpart_w_int": sigma2_operatorxpart_w_int,
            "sigma2_part": sigma2_part,
            "sigma2_operator": sigma2_operator,
            "variance_totalGRR_w_int": variance_totalGRR_w_int,
            "variance_reproducibility_w_int": variance_reproducibility_w_int,
            "var_tot_w_int": var_tot_w_int,
            "pct_var_contb_totalGRR_w_int": pct_var_contb_totalGRR_w_int,
            "pct_var_contb_repeatability_w_int": pct_var_contb_repeatability_w_int,
            "pct_var_contb_reproducibility_w_int": pct_var_contb_reproducibility_w_int,
            "pct_var_contb_operator_w_int": pct_var_contb_operator_w_int,
            "pct_var_contb_operatorxpart_w_int": pct_var_contb_operatorxpart_w_int,
            "pct_var_contb_p2p_w_int": pct_var_contb_p2p_w_int,
            "SD_totalGRR_w_int": SD_totalGRR_w_int,
            "SD_repeatability_w_int": SD_repeatability_w_int,
            "SD_reproducibility_w_int": SD_reproducibility_w_int,
            "SD_operatorxpart_w_int": SD_operatorxpart_w_int,
            "SD_total_w_int": SD_total_w_int,
            "pct_SV_totalGRR_w_int": pct_SV_totalGRR_w_int,
            "pct_SV_repeatability_w_int": pct_SV_repeatability_w_int,
            "pct_SV_reproducibility_w_int": pct_SV_reproducibility_w_int,
            "pct_SV_operator_w_int": pct_SV_operator_w_int,
            "pct_SV_operatorxpart_w_int": pct_SV_operatorxpart_w_int,
            "pct_SV_p2p_w_int": pct_SV_p2p_w_int
            }
        except Exception as e:
            print("Error", f"An error occured during ANOVA calculation: {str(e)}")
            return None


    def print_w_int(self, results):
        # Check if all necessary keys exist in the results dictionary
        necessary_keys = [
            "sigma2_w_int",
            "sigma2_operatorxpart_w_int",
            "sigma2_part",
            "sigma2_operator",
            "variance_totalGRR_w_int",
            "pct_var_contb_totalGRR_w_int",
            "SD_totalGRR_w_int",
            "pct_SV_totalGRR_w_int",
            "pct_var_contb_repeatability_w_int",
            "SD_repeatability_w_int",
            "pct_SV_repeatability_w_int",
            "variance_reproducibility_w_int",
            "pct_var_contb_reproducibility_w_int",
            "SD_reproducibility_w_int",
            "pct_SV_reproducibility_w_int",
            "sigma2_operator",
            "pct_var_contb_operator_w_int",
            "SD_operator",
            "pct_SV_operator_w_int",
            "sigma2_operatorxpart_w_int",
            "pct_var_contb_operatorxpart_w_int",
            "SD_operatorxpart_w_int",
            "pct_SV_operatorxpart_w_int",
            "sigma2_part",
            "pct_var_contb_p2p_w_int",
            "SD_p2p",
            "pct_SV_p2p_w_int",
            "var_tot_w_int"
        ]

        for key in necessary_keys:
            if key not in results:
                print(f"Error: Key '{key}' is missing in the results dictionary.")
                return

        table_data_w_int = [
            ["Sigma^2 Repeatability", results["sigma2_w_int"]],
            ["Sigma^2 Operator x Part", results["sigma2_operatorxpart_w_int"]],
            ["Sigma^2 Part", results["sigma2_part"]],
            ["Sigma^2 Operator", results["sigma2_operator"]]
        ]

        headers = ["Sigma^2", "Value"]

        print("With Interaction:")
        print(tabulate(table_data_w_int, headers=headers, tablefmt="pretty"))

        # Data for "with Interaction"
        table_data_with_int = [
            ["Total Gage R&R", results["variance_totalGRR_w_int"], results["pct_var_contb_totalGRR_w_int"], results["SD_totalGRR_w_int"], results["pct_SV_totalGRR_w_int"]],
            ["Repeatability", results["sigma2_w_int"], results["pct_var_contb_repeatability_w_int"], results["SD_repeatability_w_int"], results["pct_SV_repeatability_w_int"]],
            ["Reproducibility", results["variance_reproducibility_w_int"], results["pct_var_contb_reproducibility_w_int"], results["SD_reproducibility_w_int"], results["pct_SV_reproducibility_w_int"]],
            ["Operator", results["sigma2_operator"], results["pct_var_contb_operator_w_int"], results["SD_operator"], results["pct_SV_operator_w_int"]],
            ["Operator x Part Interaction", results["sigma2_operatorxpart_w_int"], results["pct_var_contb_operatorxpart_w_int"], results["SD_operatorxpart_w_int"], results["pct_SV_operatorxpart_w_int"]],
            ["Part to Part", results["sigma2_part"], results["pct_var_contb_p2p_w_int"], results["SD_p2p"], results["pct_SV_p2p_w_int"]],
            ["Total Variation", results["var_tot_w_int"]]
        ]

        headers = ["Source", "Variance", "% Contribution Variance", "Standard Deviation (SD)", "% Study Variance"]

        print("With Interaction:")
        print(tabulate(table_data_with_int, headers=headers, tablefmt="pretty"))


    def print_wo_int(self, results):
        # Check if all necessary keys exist in the results dictionary
        necessary_keys = [
            "sigma2_wo_int",
            "sigma2_operatorxpart_wo_int",
            "sigma2_part",
            "sigma2_operator",
            "variance_totalGRR_wo_int",
            "pct_var_contb_totalGRR_wo_int",
            "SD_totalGRR_wo_int",
            "pct_SV_totalGRR_wo_int",
            "pct_var_contb_repeatability_wo_int",
            "SD_repeatability_wo_int",
            "pct_SV_repeatability_wo_int",
            "variance_reproducibility_wo_int",
            "pct_var_contb_reproducibility_wo_int",
            "SD_reproducibility_wo_int",
            "pct_SV_reproducibility_wo_int",
            "sigma2_operator",
            "pct_var_contb_operator_wo_int",
            "SD_operator",
            "pct_SV_operator_wo_int",
            "sigma2_operatorxpart_wo_int",
            "pct_var_contb_operatorxpart_wo_int",
            "SD_operatorxpart_wo_int",
            "pct_SV_operatorxpart_wo_int",
            "sigma2_part",
            "pct_var_contb_p2p_wo_int",
            "SD_p2p",
            "pct_SV_p2p_wo_int",
            "var_tot_wo_int"
        ]

        for key in necessary_keys:
            if key not in results:
                print(f"Error: Key '{key}' is missing in the results dictionary.")
                return

        table_data_wo_int = [
            ["Sigma^2 Repeatability", results["sigma2_wo_int"]],
            ["Sigma^2 Operator x Part", results["sigma2_operatorxpart_wo_int"]],
            ["Sigma^2 Part", results["sigma2_part"]],
            ["Sigma^2 Operator", results["sigma2_operator"]]
        ]

        headers = ["Sigma^2", "Value"]

        print("Without Interaction:")
        print(tabulate(table_data_wo_int, headers=headers, tablefmt="pretty"))

        # Data for "without Interaction"
        table_data_wo_int = [
            ["Total Gage R&R", results["variance_totalGRR_wo_int"], results["pct_var_contb_totalGRR_wo_int"], results["SD_totalGRR_wo_int"], results["pct_SV_totalGRR_wo_int"]],
            ["Repeatability", results["sigma2_wo_int"], results["pct_var_contb_repeatability_wo_int"], results["SD_repeatability_wo_int"], results["pct_SV_repeatability_wo_int"]],
            ["Reproducibility", results["variance_reproducibility_wo_int"], results["pct_var_contb_reproducibility_wo_int"], results["SD_reproducibility_wo_int"], results["pct_SV_reproducibility_wo_int"]],
            ["Operator", results["sigma2_operator"], results["pct_var_contb_operator_wo_int"], results["SD_operator"], results["pct_SV_operator_wo_int"]],
            ["Operator x Part Interaction", results["sigma2_operatorxpart_wo_int"], results["pct_var_contb_operatorxpart_wo_int"], results["SD_operatorxpart_wo_int"], results["pct_SV_operatorxpart_wo_int"]],
            ["Part to Part", results["sigma2_part"], results["pct_var_contb_p2p_wo_int"], results["SD_p2p"], results["pct_SV_p2p_wo_int"]],
            ["Total Variation", results["var_tot_wo_int"]]
        ]

        headers = ["Source", "Variance", "% Contribution Variance", "Standard Deviation (SD)", "% Study Variance"]

        print("Without Interaction:")
        print(tabulate(table_data_wo_int, headers=headers, tablefmt="pretty"))

def main():
    root = tk.Tk()
    app = GaugeRRApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


+----------------------------------+----+---------------------+-------------+--------+---------+
|              Source              | DF |    Sum of Square    | Mean Square |   F    | p-value |
+----------------------------------+----+---------------------+-------------+--------+---------+
|             Operator             | 2  | 0.09992666666666687 |    0.05     |  2.0   |  0.164  |
|               Part               | 9  | 116.52942666666664  |   12.948    | 517.92 |   0.0   |
|  Interaction (operator x part)   | 18 |  0.446806666666717  |    0.025    | 0.417  |  0.979  |
| Repeatability (with Interaction) | 60 | 3.6057999999999986  |    0.06     |        |         |
|  Repeatability (wo Interaction)  | 78 | 4.0526066666667155  |    0.052    |        |         |
+----------------------------------+----+---------------------+-------------+--------+---------+
P-value: operator x part 0.979
P-value for operator x part is  0.979  which is less than threshold  0.05 . Operator x Part inte